In [349]:
#this notebook is to analyze the AlphabetSoup's donations, their past investment and predict
#which investment has the best returns to continue further investment

In [350]:
#import dependencies for loading and analyzing the data using machine learning
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [351]:
#import the csv file for data to be analyzed
charity_data_file = "Resources/charity_data.csv"
charity_df = pd.read_csv(charity_data_file, sep=",", index_col="EIN")
charity_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
EIN,,,,,,,,,,,
10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [352]:
charity_investment_df = charity_df.copy()

In [353]:
charity_investment_df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [354]:
charity_investment_df = charity_investment_df.groupby(['CLASSIFICATION']).filter(lambda x: len(x) > 2)

In [355]:
charity_investment_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
EIN,,,,,,,,,,,
10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [356]:
def ask_amount_group(ask_amt):
    if (ask_amt <= 100000):
        return 'Less than 100000'
    elif (ask_amt > 100000 and ask_amt <= 250000):
        return '100000-250000'
    elif (ask_amt > 250000 and ask_amt <= 500000):
        return '250000-500000'
    elif (ask_amt > 500000 and ask_amt <= 750000):
        return '500000-750000'
    elif (ask_amt > 750000 and ask_amt <= 1000000):
        return '750000-1000000'
    else:
        return 'Greater than 1000000'
    

In [357]:
charity_investment_df['ASK_AMT'] = charity_df['ASK_AMT'].apply(ask_amount_group)

In [358]:
charity_investment_df['ASK_AMT'].value_counts()

Less than 100000        29737
Greater than 1000000     1575
100000-250000            1440
250000-500000             862
500000-750000             386
750000-1000000            263
Name: ASK_AMT, dtype: int64

In [359]:
charity_investment_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
EIN,,,,,,,,,,,
10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,Less than 100000,1
10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,100000-250000,1
10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,Less than 100000,0
10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,Less than 100000,1
10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,100000-250000,1


In [360]:
# Create a OneHotEncoder instance to transform all of categorical data
enc = OneHotEncoder(sparse=False)

categorical_columns = [ 'APPLICATION_TYPE', 'AFFILIATION','CLASSIFICATION','INCOME_AMT', 'ASK_AMT', 'USE_CASE', 'ORGANIZATION','SPECIAL_CONSIDERATIONS']
#categorical_columns = [ 'APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION' , 'USE_CASE', 'ORGANIZATION', 'ASK_AMT', 'INCOME_AMT']
encode_df = pd.DataFrame(enc.fit_transform(charity_investment_df[categorical_columns]), index=charity_investment_df.index)

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(categorical_columns)
encode_df.head()           

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
EIN,,,,,,,,,,,,,,,,,,,,,
10520599,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
10531628,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
10547893,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
10553066,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
10556103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [361]:
#after using onehotencoder transformed data merge it with the primary data frome 
charity_investment_df = charity_investment_df.merge(encode_df, left_index=True, right_index=True)
charity_investment_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,...,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
EIN,,,,,,,,,,,,,,,,,,,,,
10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,Less than 100000,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,100000-250000,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,Less than 100000,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,Less than 100000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,100000-250000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [362]:
#drop all the columns which has transformed values
charity_investment_df.drop(columns=categorical_columns, inplace=True)

In [363]:
#drop other columns which we think of less use
#charity_investment_df.drop(columns=['USE_CASE', 'ORGANIZATION'], inplace=True)

In [364]:
#filter out charity whose status is 0 (considering they don't exist anymore)
charity_investment_df = charity_investment_df[charity_investment_df['STATUS'] == 1]

In [365]:
charity_investment_df.head()

,NAME,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
EIN,,,,,,,,,,,,,,,,,,,,,
10520599,BLUE KNIGHTS MOTORCYCLE CLUB,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
10553066,SOUTHSIDE ATHLETIC ASSOCIATION,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [366]:
#columns which might be of interest as target
target_column = 'IS_SUCCESSFUL'

In [367]:
y = charity_investment_df[target_column].values
X = charity_investment_df.drop(columns=['NAME', 'IS_SUCCESSFUL']).values

In [368]:
# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [369]:
len(X_train_scaled[0])

90

In [370]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  250
hidden_nodes_layer2 = 125
hidden_nodes_layer3 = 60
hidden_nodes_layer4 = 30
hidden_nodes_layer5 = 10


nn = tf.keras.models.Sequential()

activation_method = "relu"

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation=activation_method)
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation=activation_method))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation=activation_method))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation=activation_method))

# Fifth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation=activation_method))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])



In [ ]:
# Train the model 
fit_model = nn.fit(X_train_scaled, y_train, epochs=50) 
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25693 samples
Epoch 1/50
25693/25693 [==============================] - 6s 248us/sample - loss: 0.5663 - accuracy: 0.7230
Epoch 2/50
25693/25693 [==============================] - 5s 198us/sample - loss: 0.5513 - accuracy: 0.7307
Epoch 3/50
25693/25693 [==============================] - 5s 204us/sample - loss: 0.5475 - accuracy: 0.7334
Epoch 4/50
25693/25693 [==============================] - 5s 191us/sample - loss: 0.5452 - accuracy: 0.7332
Epoch 5/50
25693/25693 [==============================] - 5s 201us/sample - loss: 0.5426 - accuracy: 0.7340
Epoch 6/50
25693/25693 [==============================] - 5s 198us/sample - loss: 0.5425 - accuracy: 0.7333
Epoch 7/50
25693/25693 [==============================] - 5s 188us/sample - loss: 0.5408 - accuracy: 0.7370
Epoch 8/50
25693/25693 [==============================] - 5s 205us/sample - loss: 0.5397 - accuracy: 0.7370
Epoch 9/50
25693/25693 [==============================] - 5s 191us/sample - loss: 0.5391 - accuracy: 0.7367
Epoch